In [11]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
import math
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [12]:
# df loaded from google drive
df.head()

,nfl_id,opponent_coords,teammate_coords,ball_coords,game_id,play_id,frame_id,coords,pff_position_lined_up,quarter,down,yards_to_go,absolute_yardline_number,personnel_o,defenders_in_box,offense_formation,score_delta
0,41263,"[[-2.61, 0.45], [-1.29, 0.15], [0.1, -0.54], [...","[[-1.99, -2.12], [1.26, -1.79], [2.75, -2.14],...","[0, 0]",2021090900,97,6,"[-5.03, -1.74]",LEO,1,3,2,43,"1 RB, 1 TE, 3 WR",6.0,SHOTGUN,0
1,42403,"[[3, 0.18], [1.73, -0.37], [0.1, -0.54], [-1.2...","[[2.75, -2.14], [1.26, -1.79], [-1.99, -2.12],...","[0, 0]",2021090900,97,6,"[8.71, -2.34]",ROLB,1,3,2,43,"1 RB, 1 TE, 3 WR",6.0,SHOTGUN,0
2,44955,"[[1.73, -0.37], [0.1, -0.54], [3, 0.18], [-1.2...","[[2.75, -2.14], [-1.99, -2.12], [-5.03, -1.74]...","[0, 0]",2021090900,97,6,"[1.26, -1.79]",DRT,1,3,2,43,"1 RB, 1 TE, 3 WR",6.0,SHOTGUN,0
3,53441,"[[-1.29, 0.15], [0.1, -0.54], [-2.61, 0.45], [...","[[-5.03, -1.74], [1.26, -1.79], [2.75, -2.14],...","[0, 0]",2021090900,97,6,"[-1.99, -2.12]",LILB,1,3,2,43,"1 RB, 1 TE, 3 WR",6.0,SHOTGUN,0
4,53504,"[[1.73, -0.37], [3, 0.18], [0.1, -0.54], [-1.2...","[[1.26, -1.79], [-1.99, -2.12], [8.71, -2.34],...","[0, 0]",2021090900,97,6,"[2.75, -2.14]",RE,1,3,2,43,"1 RB, 1 TE, 3 WR",6.0,SHOTGUN,0


In [13]:
def split_sequence(sequence, n_steps):
    X, y = list(), list()
    for i in range(len(sequence)):
        end_ix = i + n_steps
        if end_ix > len(sequence)-1:
            break
        seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
        X.append([[c[0] for c in seq_x], [c[1] for c in seq_x]])
        y.append(seq_y)
    return X, y

In [14]:
def make_features(df, n_steps):
    X, y  = list(), list()
    grouped_df = df.groupby(["nfl_id","game_id","play_id"])
    for group_name, group_df in grouped_df:
        coords = group_df["coords"].tolist()
        X1, y1 = split_sequence(coords, n_steps)
        X += X1
        y += y1
    return np.array(X), np.array(y)
    # return pad_sequences(X, value = -99, dtype= "float"), pad_sequences(y, value = -99, dtype= "float")


In [15]:
# the dataset is as follows

# each observation is contains an array of length n_steps*2 containing x and y coords from
# the previous n_steps and an array of length 2 containing the current x and y coord pair
# thus, we are predicting current frame position based in previous n_steps frame. there is no 
# separation between videos

X, y = make_features(df, 5)

# train on 50k frames
X_train = X[:50000]
y_train = y[:50000]

# test on 10k frames
X_test = X[-10000:]
y_test = y[-10000:]

In [16]:
# set random seed for reproducibility
tf.random.set_seed(42)
np.random.seed(42)

# create the model
model = tf.keras.Sequential()
# model.add(tf.keras.layers.TimeDistributed(tf.keras.layers.Flatten(), input_shape=(96, 2, 3)))
model.add(tf.keras.layers.LSTM(32, input_shape=(2, 5)))
model.add(tf.keras.layers.Dense(2))

# compile and fit the model
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(X_train, y_train, epochs=100, verbose=0)

2022-12-30 17:52:04.360315: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [18]:
err = 0
for i,j in zip(model.predict(X_test), y_test):
    err += math.dist(i,j)
print(err/len(y_test)) # avg prediction error (yards)

313/313 [==============================] - 1s 3ms/step
0.037320989691664724
